# KBMOD Search Results for Fakes
  
A basic notebook to demonstrate searching results against known fakes in the data.

Note that this notebook is intended to be run using shared data on baldur.

# Setup demo

Before importing, make sure you have installed kbmod using `pip install .` in the root `KBMOD` directory.  Also be sure you are running with python3 and using the correct notebook kernel.

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os

from kbmod.analysis.plotting import *
from kbmod.data_interface import load_deccam_layered_image
from kbmod.search import ImageStack, PSF, StampCreator, Trajectory
from kbmod.results import Results
from kbmod.work_unit import WorkUnit

from astropy.coordinates import SkyCoord, search_around_sky
import astropy.units as u
from astropy.table import Table

# Data paths
wu_path = '/epyc/projects/kbmod/runs/wbeebe/1000_imgs_5_16/reprojected_wu.fits' # A reflex-corrected WorkUnit
res_path = '/epyc/projects/kbmod/runs/wbeebe/1000_imgs_5_16_slow'
# Path to known fakes (with reflex-corrected) coordinates on the dates used in this KBMOD search.
fakes_path = '/epyc/projects/kbmod/runs/wbeebe/fakes_detections_20190404_20190505_simple.csv'

# Reload a Saved KBMOD WorkUnit
Note that this WorkUnit was reflex-corrected with a guess distance of 40 AU.

In [ ]:
wu = WorkUnit.from_fits(wu_path) 
stack = wu.im_stack

# Load the KBMOD Results and Known Fakes

In [ ]:
results = Results.read_table(os.path.join(res_path, "results.ecsv"))
print(f"Loaded {len(results)} results.")
results

And now load our known fakes

In [ ]:
fakes = Table.read(fakes_path, format='csv')
fakes

# Search for results that are near known fakes

astropy allows us to take two catalogus of coordinates (represented by `SkyCoord` objects) and easily search for nearest neighbors between them. 

First, as a simple approximation let's translate the initial (x, y) of each of our results into an (ra, dec). Note that we are using a reflex-corrected WCS from our `WorkUnit` so the (ra, dec) will be in reflex-corrected space.

In [ ]:
results_ra_dec = [wu.wcs.pixel_to_world(r["x"], r["y"]) for r in results]
results.table["ra_dec"] = results_ra_dec
results

Now we can translate our (ra, dec) pairs into single `SkyCoord` objects.

Then we can use astopy's `search_around_sky` to find which KBMOD results are near our known fakes, with a max separation limit of 1 arcsecond

In [ ]:
result_coords = SkyCoord(ra=[coord.ra for coord in results_ra_dec], dec=[coord.dec for coord in results_ra_dec])

# Since our WorkUnit was reflex-corrected with a guess distance of 40 AU,
# we use the corresponding reflex-correced (ra, dec) for our fakes.
fake_coords = SkyCoord(ra=fakes["RA_40"]*u.degree, dec=fakes["Dec_40"]*u.degree)

idx1, idx2, sep2dAngle, dist3d = search_around_sky(result_coords, fake_coords, 1*u.arcsecond)

`idx1` Maps fakes to their nearest neighbors in the results. Each value is an index to a potential finding within our results table.

In [ ]:
print(len(idx1))
idx1

In [ ]:
potential_fake_results = np.unique(idx1)
potential_fake_results

Let's first examine the first result which we believe might be a fake.

In [ ]:
results[potential_fake_results[0]]

In [ ]:
# helper function to plot a row of the results table
plot_result_row(results[potential_fake_results[0]])

`idx2` shows the inverse mapping of which fakes might be potential results. Here its values are indices within our fakes table.

So taking the first potential match we examined above, we can use the corresponding index (in this case 0) to inspect within our fakes table.


In [ ]:
fakes[idx2[0]]

And as a sanity check we can also inspect the distance between the result and fake in reflex-corrected space.

In [ ]:
dist3d[0]

# Plot the Coadds of all Suspected Fakes within Our Results

In [ ]:
# Plot the coadds of all potential fakes
for idx in potential_fake_results:
    # helper function to plot a row of the results table
    plot_result_row(results[idx])